In [1]:
import numpy as np
import pandas as pd
import re

import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest

C:\Users\smvan\anaconda3\lib\site-packages\pdpipe\col_generation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
consolidated_df = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/Consolidated_purchases/OK_consol_purchases_march1_april21.csv")
consolidated_df.head()

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,prod_group_descrip,order_code,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount
0,252033,M123033004,-172.0,69025,-59.0,-123.9,-123.9,each,2.1,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
1,252099,0252099,153.3,69025,15.0,31.5,31.5,each,2.1,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
2,252100,0252099,-153.3,69025,-15.0,-31.5,-31.5,each,2.1,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
3,252099,0252099,153.3,68933,14.0,29.4,29.4,each,2.1,CANDIED COCONUT ALMO,...,CANDIES SNACKS,KK051093,N,1.0,1.0,each,7,Candy,N,0
4,252100,0252099,-153.3,68933,-14.0,-29.4,-29.4,each,2.1,CANDIED COCONUT ALMO,...,CANDIES SNACKS,KK051093,N,1.0,1.0,each,7,Candy,N,0


In [3]:
categories_df = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/GHG_categories/ghge_factors.csv")
categories_df


,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316
...,...,...,...
58,59,manually adjusted,0.0000
59,60,human labor,0.0000
60,61,kitchen supplies,0.0000
61,62,mushrooms,1.5000


In [4]:
# Convert the "item_descrip" and "Food Category" columns to lowercase
consolidated_df['item_descrip'] = consolidated_df['item_descrip'].str.lower()
categories_df['Food Category'] = categories_df['Food Category'].str.lower()

# Initialize new columns
consolidated_df['Food Category'] = np.nan
consolidated_df['Category ID'] = np.nan

# Define a function to split the Food Category column
# def split_food_category(category):
#     separators = ['/', '(', ',', ')', '|']
#     for separator in separators:
#         category = category.replace(separator, ' ')
#     return category.strip()

def split_food_category(category):
    separators = ['/', '(', ',', ')', '|']
    for separator in separators:
        category = category.replace(separator, ' ')
    return category.strip()

for index, row in categories_df.iterrows():
    categories_df.at[index, 'Food Category'] = split_food_category(row['Food Category'])
    
for index, row in consolidated_df.iterrows():
    consolidated_df.at[index, 'item_descrip'] = split_food_category(row['item_descrip'])
    
    
# Iterate through each row
for index, row in consolidated_df.iterrows():
    item_descrip = row['item_descrip']
    
    # Split the item_descrip into words
    item_words = item_descrip.split()
    
    # Find matches between the words in item_descrip and the Food Category column
    matches = categories_df[categories_df['Food Category'].apply(lambda x: any(word in x.split() for word in item_words))]
    
    # If there is a match, assign the corresponding values
    if not matches.empty:
        consolidated_df.at[index, 'Food Category'] = matches['Food Category'].iloc[0]
        consolidated_df.at[index, 'Category ID'] = matches['Category ID'].iloc[0]

# Print the updated DataFrame
consolidated_df





,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID
0,252033,M123033004,-172.00,69025,-59.0,-123.90,-123.90,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,almond milk,27.0
1,252099,0252099,153.30,69025,15.0,31.50,31.50,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,almond milk,27.0
2,252100,0252099,-153.30,69025,-15.0,-31.50,-31.50,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,almond milk,27.0
3,252099,0252099,153.30,68933,14.0,29.40,29.40,each,2.10,candied coconut almo,...,N,1.0,1.0,each,7,Candy,N,0,NaN,NaN
4,252100,0252099,-153.30,68933,-14.0,-29.40,-29.40,each,2.10,candied coconut almo,...,N,1.0,1.0,each,7,Candy,N,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18381,252537,9001537618,3899.87,5528,4.0,208.92,208.92,cs,52.23,wrap film roll 17x2500 ct box,...,N,2500.0,1.0,cs,4,Misc.,N,0,NaN,NaN
18382,252458,9001537197,7606.70,5528,1.0,52.23,52.23,cs,52.23,wrap film roll 17x2500 ct box,...,N,2500.0,1.0,cs,4,Misc.,N,0,NaN,NaN
18383,250022,9001206712,8283.19,30507,3.0,225.57,225.57,cs,75.19,"wrap foil hd 18"" 45cmx100m",...,N,328.0,1.0,cs,4,Misc.,N,0,NaN,NaN
18384,250792,9001333265,2352.89,30507,1.0,75.19,75.19,cs,75.19,"wrap foil hd 18"" 45cmx100m",...,N,328.0,1.0,cs,4,Misc.,N,0,NaN,NaN


In [5]:
pd.DataFrame(consolidated_df.iloc[15644, :])

,15644
invoice_counter,251831
invoice_num,00023247
invoice_total,301.21
item_num,67291
qty,1.0
line_amount,12.0
adj_line_amount,12.0
uom,ea
unit_cost,12.0
item_descrip,ck easter cookeis egg 6 ea


In [6]:
# for index, row in categories_df.iterrows():
#     for i, r in consolidated_df.iterrows():
#         if consolidated_df.loc[i, 'Food Category'].str.contains(categories_df.loc[index, 'Food Category']):
#             consolidated_df.loc[index, 'Food Category'] = categories_df.loc[index, 'Food Category']
#             consolidated_df.loc[index, 'Category ID'] = categories_df.loc[index, 'Category ID']

In [7]:
# def split_food_category(category):
#     separators = ['/', '(', ',', ')']
#     for separator in separators:
#         category = category.replace(separator, ' ')
#     return category.strip()

# for index, row in categories_df.iterrows():
#     categories_df.at[index, 'Food Category'] = split_food_category(row['Food Category'])

a_df = consolidated_df.iloc[730:760, :]


In [8]:
# Iterate through each row
for index, row in consolidated_df.iterrows():
    item_descrip = row['item_descrip']
    
    # Split the item_descrip into words
    item_words = item_descrip.split()
    
    if 'eggplant' in item_words or 'kale' in item_words or 'lettuce' in item_words or 'romaine' in item_words or 'salad':
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
        
    if 'base' in item_words and 'veg' in item_words or 'veg' in item_words or 'seaweed' in item_words or 'ck|g&g|salad|kalecaesarw/chick' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
        
    if 'nori' in item_words or 'ck|g&g|salad|kale caesar.' in item_words or 'pickle' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40

        
    if 'spinach' in item_words or 'artichoke' in item_words or 'avocado' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
        
    if 'cantaloupe' in item_words or 'papaya' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'Category ID'] = 31
    
    if 'celery' in item_words or 'coleslaw' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
    
    if 'sprouts' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
    
    if 'milkettes' in item_words or ('milk' in item_words and '2%' in item_words) or 'milk' in item_words:
        consolidated_df.at[index, 'Food Category'] = "milk (cow's milk)"
        consolidated_df.at[index, 'Category ID'] = 9
    
    
    if ('milk' in item_words and 'rice' in item_words):
        consolidated_df.at[index, 'Food Category'] = "rice milk"
        consolidated_df.at[index, 'Category ID'] = 29
        
    if 'apples' in item_words or 'apple' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'apples'
        consolidated_df.at[index, 'Category ID'] = 32            
    
    if 'oat' in item_words or 'oatmeal' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'oats (oatmeal)'
        consolidated_df.at[index, 'Category ID'] = 23

    if 'dress' in item_words or 'sauce' in item_words or 'jelly' in item_words or 'ketchup' in item_words or 'chutney' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sauces & paste'
        consolidated_df.at[index, 'Category ID'] = 58

    if 'mayonaise*vegan' in item_words or 'mayonnaise' in item_words or 'mustard' in item_words or 'spread' in item_words or 'paste' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sauces & paste'
        consolidated_df.at[index, 'Category ID'] = 58
        
        

    # Check if item_descrip contains "cereal"
    if 'cereal' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other grains/cereals'
        consolidated_df.at[index, 'Category ID'] = 21
    

    
    # Check if item_descrip contains "salami"
    if 'salami' in item_words or '50/50' in item_words or 'patties' in item_words or ('beef' in item_words and 'roast' in item_words) or 'beef' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'beef & buffalo meat'
        consolidated_df.at[index, 'Category ID'] = 1
              

    if 'coconut' in item_words or 'apricot' in item_words or 'cucumber' in item_words or 'peppers' in item_words or 'salsa' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'Category ID'] = 31 

    if 'grapes' in item_words or 'kiwi' in item_words or 'mangoes' in item_words or 'honeydew' in item_words or 'peach' in item_words or 'pears' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'Category ID'] = 31 
    
    if 'squash' in item_words or 'watermelon' in item_words or 'zucchini' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'Category ID'] = 31 
        
    if 'muffin' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24     

        
    if ('cream' in item_words and 'ice' in item_words) or 'magnum' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'ice cream'
        consolidated_df.at[index, 'Category ID'] = 7  
        
        
    if 'pie' in item_words or 'waffle' in item_words or 'pastry' in item_words or 'flour' in item_words or 'bun' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
    
    if 'bar' in item_words or 'biscuits' in item_words or 'brownie' in item_words or 'cake' in item_words or 'cookie' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
        
    
    if 'loaf' in item_words or 'pretzel' in item_words or 'scone' in item_words or 'tart' in item_words or 'tiramisu' in item_words or 'lady' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24

    if 'crouton' in item_words or 'ck|prep|granola' in item_words or 'noodles' in item_words or 'perogy' in item_words or 'pasta' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
    
               
#     if 'co2' in item_words or 'soda' in item_words or 'baking' in item_words:
#         consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
#         consolidated_df.at[index, 'Category ID'] = 61
            
        
    if 'hashbrown' in item_words or 'potato*pompom' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'potatoes'
        consolidated_df.at[index, 'Category ID'] = 41
    
    if 'dragon' in item_words or 'california' in item_words or 'shrimpmt' in item_words or 'prawn' in item_words or 'shrimp' in item_words or 'prawns' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'crustaceans (shrimp/prawns)'
        consolidated_df.at[index, 'Category ID'] = 13


    if 'cumin' in item_words or 'cloves' in item_words or 'ground' in item_words or 'seasoning' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54
        
    if 'blend' in item_words or 'chili' in item_words or 'pwdr' in item_words or 'caper' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54
    
    if 'leaf' in item_words or 'chile' in item_words or 'paprika' in item_words or 'leaves' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54   
        
    if 'anise' in item_words or 'cardamom' in item_words or 'mix' in item_words or 'wasabi' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54 
        
        
    if 'onion' in item_words or 'leeks' in item_words or 'onions' in item_words or 'shallots' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'onions and leeks'
        consolidated_df.at[index, 'Category ID'] = 39
                
    
    if 'pumpkin' in item_words or 'jackfruit' in item_words or 'pineapple' in item_words or 'raisin' in item_words or 'cherries' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'Category ID'] = 31 
        
    if 'pumpkin' in item_words and 'kern' in item_words or 'quinoa' in item_words or 'sesame' in item_words or 'seed' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'tree nuts and seeds'
        consolidated_df.at[index, 'Category ID'] = 26 
        

    if 'tomato' in item_words or 'tomatillos' in item_words or 'tomatoes' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'tomatoes'
        consolidated_df.at[index, 'Category ID'] = 37
        
    if 'soybean' in item_words or 'tofu' in item_words or 'tempeh' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'soybeans/tofu'
        consolidated_df.at[index, 'Category ID'] = 20
    
    if 'sake' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wine grapes (wine)'
        consolidated_df.at[index, 'Category ID'] = 51
    
    if 'lentils' in item_words or 'ck|g&g|salad|medchickpea' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other legumes'
        consolidated_df.at[index, 'Category ID'] = 16
    
    if 'chickpea.' in item_words or 'falafel' in item_words or ('patties' in item_words and 'vegan' in item_words):
        consolidated_df.at[index, 'Food Category'] = 'other legumes'
        consolidated_df.at[index, 'Category ID'] = 16
    
    
    if 'tuna' in item_words or 'squid' in item_words or 'cod' in item_words or 'halibut' in item_words or 'salmon' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'fish (finfish)'
        consolidated_df.at[index, 'Category ID'] = 12
    
    if 'raspberry' in item_words or 'blackberry' in item_words or 'blueberry' in item_words or 'strawberry' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'berries'
        consolidated_df.at[index, 'Category ID'] = 34
        
    if 'raspberries' in item_words or 'blackberries' in item_words or 'blueberries' in item_words or 'strawberries' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'berries'
        consolidated_df.at[index, 'Category ID'] = 34
    
    if 'sal' in item_words or 'samosa' in item_words or 'wrap' in item_words or 'sandwich' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'manually adjusted'
        consolidated_df.at[index, 'Category ID'] = 59
        
    if 'coconut' in item_words and 'milk' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'manually adjusted'
        consolidated_df.at[index, 'Category ID'] = 59
 
    if 'mandarin' in item_words or 'grapefruit' in item_words or 'lemons' in item_words or 'limes' in item_words or 'mandarins' in item_words or 'oranges' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'citrus fruit'
        consolidated_df.at[index, 'Category ID'] = 35
    
    if 'olive' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'olives (oil)'
        consolidated_df.at[index, 'Category ID'] = 49
    
    if 'beets' in item_words or 'dill' in item_words or 'ginger' in item_words or 'garlic' in item_words or 'jicama' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'root vegetables'
        consolidated_df.at[index, 'Category ID'] = 38
   
    if 'parsnip' in item_words or 'carrot' in item_words or 'carrots' in item_words or 'turnip' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'root vegetables'
        consolidated_df.at[index, 'Category ID'] = 38
        
        
    if 'thyme' in item_words or 'herb' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'herbs'
        consolidated_df.at[index, 'Category ID'] = 63    
        
    if 'pea' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'peas'
        consolidated_df.at[index, 'Category ID'] = 18 
    
    if 'yogurt' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'yogurt'
        consolidated_df.at[index, 'Category ID'] = 10
        
    if 'pumpkin' in item_words or 'hemp' in item_words or 'nut' in item_words or 'almond' in item_words or 'seeds' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'tree nuts and seeds'
        consolidated_df.at[index, 'Category ID'] = 26
    
    if 'crepe' in item_words or 'croissant' in item_words or 'danish' in item_words or 'donut' in item_words or 'eclair' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
    
        # Check if prod_group_descrip is "SUPPLY"
#     if row['prod_group_descrip'] == 'SUPPLY':
#         consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
#         consolidated_df.at[index, 'Category ID'] = 61
        
#     if row['prod_group_descrip'] == 'DISPOSABLES':
#         consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
#         consolidated_df.at[index, 'Category ID'] = 61

#     if row['prod_group_descrip'] == 'CLEANING':
#         consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
#         consolidated_df.at[index, 'Category ID'] = 61
        
    if row['prod_group_descrip'] == 'BREAD':
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24   
    
    if row['prod_group_descrip'] == 'BAKED GOODS':
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
        
    if row['prod_group_descrip'] == 'CANDIES SNACKS':
        consolidated_df.at[index, 'Food Category'] = 'sugars and sweeteners'
        consolidated_df.at[index, 'Category ID'] = 43
    
    if row['prod_group_descrip'] == 'BEVERAGE' and pd.isna(row['Food Category']):
        consolidated_df.at[index, 'Food Category'] = 'water & beverages'
        consolidated_df.at[index, 'Category ID'] = 55
    
    if 'pringles' in item_words or 'pringle' in item_words or 'potato' in item_words or 'fries' in item_words or 'chip' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'potatoes'
        consolidated_df.at[index, 'Category ID'] = 41
    
    if ('base' in item_words and 'chick' in item_words) or ('thigh' in item_words) or ('wings' in item_words) or 'chicken' in item_words or 'turkey' in item_words or 'chick' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'poultry (chicken, turkey)'
        consolidated_df.at[index, 'Category ID'] = 4
    
        # Check if item_descrip contains "proscuitto" or "ham"
    if 'prosciutto' in item_words or 'ham' in item_words or 'bacon' in item_words or 'chorizo' in item_words or 'sausage' in item_words or 'pork' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'pork (pig meat)'
        consolidated_df.at[index, 'Category ID'] = 3

    if 'juice' in item_words or 'pop' in item_words or 'coke' in item_words or 'ale' in item_words or 'inst' in item_words or 'tea' in item_words or 'bev' in item_words or 'punch' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'water & beverages'
        consolidated_df.at[index, 'Category ID'] = 55 
        
    if 'water' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'water & beverages'
        consolidated_df.at[index, 'Category ID'] = 55 
 
    if 'rubbed' in item_words or 'pepper' in item_words or 'powder' in item_words or 'masala' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54
        
    if row['prod_group_descrip'] == 'SUPPLY':
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'Category ID'] = 61
        
    if row['prod_group_descrip'] == 'DISPOSABLES':
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'Category ID'] = 61

    if row['prod_group_descrip'] == 'CLEANING':
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'Category ID'] = 61
        
    
    if 'beer' in item_words:
        consolidated_df.at[index, 'Food Category'] = "barley (beer)"
        consolidated_df.at[index, 'Category ID'] = 50
    
    if 'sugar' in item_words or 'melona' in item_words or 'honey' in item_words or 'syrup' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sugars and sweeteners'
        consolidated_df.at[index, 'Category ID'] = 43 
        

    if 'salt&vin' in item_words or 'balsamic' in item_words or 'vinegar' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'vinegar'
        consolidated_df.at[index, 'Category ID'] = 57
    
    if 'salt' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'salt'
        consolidated_df.at[index, 'Category ID'] = 56
        
    if 'butter' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'butter'
        consolidated_df.at[index, 'Category ID'] = 5
          
        
    if ('cream' in item_words or 'creamo10%' in item_words) and not 'ice' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cream'
        consolidated_df.at[index, 'Category ID'] = 8  
        
    if 'paneer' in item_words or 'cheese' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cheese'
        consolidated_df.at[index, 'Category ID'] = 6
    
    
    if 'egg' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'eggs'
        consolidated_df.at[index, 'Category ID'] = 11
        
    
    if ('milk' in item_words and 'go' in item_words) or ('milk' in item_words and '2%' in item_words) or ('milk' in item_words and 'choc' in item_words):
        consolidated_df.at[index, 'Food Category'] = "milk (cow's milk)"
        consolidated_df.at[index, 'Category ID'] = 9
        
    if 'milk' in item_words and 'homo' in item_words:
        consolidated_df.at[index, 'Food Category'] = "milk (cow's milk)"
        consolidated_df.at[index, 'Category ID'] = 9

    if ('milk' in item_words and 'soy' in item_words):
        consolidated_df.at[index, 'Food Category'] = "soy milk"
        consolidated_df.at[index, 'Category ID'] = 30
        
    if ('milk' in item_words and 'oat' in item_words) or ('oat' in item_words and 'barista' in item_words):
        consolidated_df.at[index, 'Food Category'] = "oat milk"
        consolidated_df.at[index, 'Category ID'] = 28
    
    if 'banana' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'bananas'
        consolidated_df.at[index, 'Category ID'] = 33
    
    if 'mushrooms' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'mushrooms'
        consolidated_df.at[index, 'Category ID'] = 62
        
    if 'corn' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'corn (maize)'
        consolidated_df.at[index, 'Category ID'] = 22
    
    if 'lamb' in item_words or 'mutton' in item_words or 'goat' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'lamb/mutton & goat meat'
        consolidated_df.at[index, 'Category ID'] = 2
        
        
    if 'broccoli' in item_words or 'cabbage' in item_words or 'cauliflower' in item_words or 'kale' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cabbages and other brassicas (broccoli)'
        consolidated_df.at[index, 'Category ID'] = 36   

    if 'daikon' in item_words or 'brussel' in item_words or 'choy' in item_words or 'broccolini' in item_words or 'gai' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cabbages and other brassicas (broccoli)'
        consolidated_df.at[index, 'Category ID'] = 36 

    if 'co2' in item_words or 'soda' in item_words or 'baking' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'Category ID'] = 61
    
    if 'easter' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sugars and sweeteners'
        consolidated_df.at[index, 'Category ID'] = 43
        
    if 'oil' in item_words or 'oils' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetable oils'
        consolidated_df.at[index, 'Category ID'] = 44

    if ('oil' in item_words and 'soybeans' in item_words) or ('oil' in item_words and 'soybean' in item_words) or ('oil' in item_words and 'soy' in item_words):
        consolidated_df.at[index, 'Food Category'] = 'soybeans (oil)'
        consolidated_df.at[index, 'Category ID'] = 45
    
    if 'oil' in item_words and 'palm' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'palm (oil)'
        consolidated_df.at[index, 'Category ID'] = 46

    if 'oil' in item_words and 'sunflower' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sunflower (oil)'
        consolidated_df.at[index, 'Category ID'] = 47

    if ('oil' in item_words and 'rapeseed' in item_words) or ('oil' in item_words and 'canola' in item_words):
        consolidated_df.at[index, 'Food Category'] = 'rapeseed/canola (oil)'
        consolidated_df.at[index, 'Category ID'] = 48
        
    if 'bean' in item_words or 'beans' in item_words or 'pulses' in item_words or 'chana' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'beans and pulses (dried)'
        consolidated_df.at[index, 'Category ID'] = 17
    
    
    if 'chocolate' in item_words or 'arnotts' in item_words or 'choco' in item_words or 'cho' in item_words or 'cocoa' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cocoa'
        consolidated_df.at[index, 'Category ID'] = 52
        
    if 'dough' in item_words or 'cracker' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
    
    if 'rice' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'rice'
        consolidated_df.at[index, 'Category ID'] = 25
    
# consolidated_df
# 18386

#     elif 'chip' in item_words or 'potato' in item_words:
#         consolidated_df.at[index, 'Food Category'] = 'potatoes'
#         consolidated_df.at[index, 'Category ID'] = 41
# Print the updated DataFrame

In [9]:
#a_df = consolidated_df.iloc[421:41, :]
a_df = consolidated_df.iloc[14043:14070, :]
a_df

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID
14043,247723,1061818080,2562.75,17159,2.0,74.30,74.30,cs,37.15,tomatoes grape 10lb,...,N,1.0,10.0,cs,2,Food,Y,0,tomatoes,37.0
14044,247867,1061818085,193.65,17159,1.0,37.15,37.15,cs,37.15,tomatoes grape 10lb,...,N,1.0,10.0,cs,2,Food,Y,0,tomatoes,37.0
14045,248142,1061818275,252.70,17159,2.0,8.50,8.50,pak,4.25,tomatoes grape 10lb,...,N,1.0,10.0,cs,2,Food,N,0,tomatoes,37.0
14046,247966,1061818272,3124.20,17159,5.0,185.75,185.75,cs,37.15,tomatoes grape 10lb,...,N,1.0,10.0,cs,2,Food,Y,0,tomatoes,37.0
14047,247868,1061818276,252.80,17159,1.0,37.15,37.15,cs,37.15,tomatoes grape 10lb,...,N,1.0,10.0,cs,2,Food,Y,0,tomatoes,37.0
14048,247768,1061818515,216.40,17159,1.0,37.15,37.15,cs,37.15,tomatoes grape 10lb,...,N,1.0,10.0,cs,2,Food,Y,0,tomatoes,37.0
14049,248312,1061818519,4630.10,17159,7.0,260.05,260.05,cs,37.15,tomatoes grape 10lb,...,N,1.0,10.0,cs,2,Food,Y,0,tomatoes,37.0
14050,247970,1061818723,1551.95,17159,8.0,297.20,297.20,cs,37.15,tomatoes grape 10lb,...,N,1.0,10.0,cs,2,Food,Y,0,tomatoes,37.0
14051,247968,1061818520,3416.00,17159,2.0,74.30,74.30,cs,37.15,tomatoes grape 10lb,...,N,1.0,10.0,cs,2,Food,Y,0,tomatoes,37.0
14052,248096,1061818912,173.45,17159,1.0,37.15,37.15,cs,37.15,tomatoes grape 10lb,...,N,1.0,10.0,cs,2,Food,Y,0,tomatoes,37.0


In [10]:
# def split_food_category(category):
#     separators = ['/', '(', ',', ')']
#     for separator in separators:
#         category = category.replace(separator, ' ')
#     return category.strip()

# for index, row in categories_df.iterrows():
#     categories_df.at[index, 'Food Category'] = split_food_category(row['Food Category'])
    
# # Iterate through each row
# for index, row in consolidated_df.iterrows():
#     item_descrip = row['item_descrip']
    
#     # Check if "Food Category" is NaN
#     if pd.isna(row['Food Category']):
    
#         # Split the item_descrip into words
#         item_words = item_descrip.split()
        
#         # Find matches between the words in item_descrip and the Food Category column
#         matches = categories_df[categories_df['Food Category'].apply(lambda x: any(word in x.split() for word in item_words))]
        
#         # If there is a match, assign the corresponding values
#         if not matches.empty:
#             consolidated_df.at[index, 'Food Category'] = matches['Food Category'].iloc[0]
#             consolidated_df.at[index, 'Category ID'] = matches['Category ID'].iloc[0]

# # Print the updated DataFrame
# consolidated_df

In [11]:
filtered_df = consolidated_df[~consolidated_df["Category ID"].isna()]
filtered_df

# filtered_df = a_df[~a_df["Category ID"].isna()]
# filtered_df


,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID
0,252033,M123033004,-172.00,69025,-59.0,-123.90,-123.90,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
1,252099,0252099,153.30,69025,15.0,31.50,31.50,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
2,252100,0252099,-153.30,69025,-15.0,-31.50,-31.50,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
3,252099,0252099,153.30,68933,14.0,29.40,29.40,each,2.10,candied coconut almo,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
4,252100,0252099,-153.30,68933,-14.0,-29.40,-29.40,each,2.10,candied coconut almo,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18381,252537,9001537618,3899.87,5528,4.0,208.92,208.92,cs,52.23,wrap film roll 17x2500 ct box,...,N,2500.0,1.0,cs,4,Misc.,N,0,kitchen supplies,61.0
18382,252458,9001537197,7606.70,5528,1.0,52.23,52.23,cs,52.23,wrap film roll 17x2500 ct box,...,N,2500.0,1.0,cs,4,Misc.,N,0,kitchen supplies,61.0
18383,250022,9001206712,8283.19,30507,3.0,225.57,225.57,cs,75.19,"wrap foil hd 18"" 45cmx100m",...,N,328.0,1.0,cs,4,Misc.,N,0,kitchen supplies,61.0
18384,250792,9001333265,2352.89,30507,1.0,75.19,75.19,cs,75.19,"wrap foil hd 18"" 45cmx100m",...,N,328.0,1.0,cs,4,Misc.,N,0,kitchen supplies,61.0


In [12]:
consolidated_df[(consolidated_df['Category ID'] == 40.0) & (~consolidated_df['item_num'].duplicated(keep=False))]

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID
6036,251531,9001425820,8341.31,63034,2.0,123.78,123.78,cs,61.89,burger veg malibu gardenburger,...,N,1.00,48.0,cs,2,Food,Y,0,other vegetables,40.0
6596,250453,1061822848,1301.85,30178,1.0,25.85,25.85,pak,25.85,nori sheets,...,N,1.00,100.0,pak,2,Food,N,0,other vegetables,40.0
7983,248126,9001081628,504.98,2716,1.0,49.03,49.03,cs,49.03,soup chowder clam boston,...,N,1.81,3.0,cs,2,Food,N,0,other vegetables,40.0
8405,250390,2043233,5728.31,11587,8.0,75.60,75.60,each,9.45,wakame cut,...,N,16.00,1.0,CT,2,Food,N,0,other vegetables,40.0
8865,247877,9001094408,7899.82,10503,4.0,344.56,344.56,cs,86.14,hot dog plant based wiener fzn,...,N,4.00,1.0,cs,2,Food,N,0,other vegetables,40.0
10561,249863,1061820707,1651.70,4606,1.0,77.55,77.55,cs,77.55,celery roothl,...,N,1.00,25.0,cs,2,Food,N,0,other vegetables,40.0
10839,250199,1061822560,2035.35,4642,1.0,93.45,93.45,cs,93.45,eggplant japanesemx,...,N,1.00,30.0,cs,2,Food,Y,0,other vegetables,40.0
11655,249828,IN162549,785.59,17932,4.0,248.44,248.44,pak,62.11,kelp konbu,...,N,1.00,1.0,pak,2,Food,Y,0,other vegetables,40.0
11948,248479,GP03-06-2023,105.00,69726,-5.0,-15.00,-15.00,HEAD,3.00,lettuce red looseleaf,...,N,1.00,1.0,HEAD,2,Food,N,0,other vegetables,40.0


In [13]:
consolidated_df.to_csv('C:/Users/smvan/CFFS-S23/CFFS-22-23/data/GHG_categories/GHG_assigned_data.csv', index = False)

In [14]:
unfiltered_df = consolidated_df[consolidated_df["Category ID"].isna()]
unfiltered_df

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID


**Unit conversions**

Finding out percentage of items would need manual adjustments to units and don’t come in standard units like kg, lbs, oz, ml, etc.

In [15]:
# consolidated_df['uom'].unique()

In [16]:
# consolidated_df['case_uom'].unique()

In [17]:
# import numpy as np

# # Convert measurements to lowercase
# unique_uom = consolidated_df['uom'].str.lower().unique()

# # Calculate % of non-standard units
# standard_units = ['kg', 'lbs', 'oz', 'ml']
# non_standard_count = np.sum([uom not in standard_units for uom in unique_uom])
# percentage_non_standard = (non_standard_count / len(unique_uom)) * 100

# print(f"Percentage of items needing manual adjustments to units: {percentage_non_standard:.2f}%")


In [18]:
# # Convert case_uom values to lowercase
# unique_case_uom = consolidated_df['case_uom'].str.lower().unique()

# # Calculate % of non-standard case units
# standard_units = ['case', 'box', 'pack']
# non_standard_count = np.sum([case_uom not in standard_units for case_uom in unique_case_uom])
# percentage_non_standard = (non_standard_count / len(unique_case_uom)) * 100

# print(f"Percentage of items needing manual adjustments to case units: {percentage_non_standard:.2f}%")


In [19]:
# # Read Coversions_List.csv that was created from 1_data preprocessing
# Conversions = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/preprocessed/Conversions_List.csv")
# Conversions.dtypes

In [20]:
# Standard unit converter:
# Construct a unit converter for specific ingredients

# After this line below, spc_cov contains only the non-empty values from the 'ConversionId' column of the Conversions DataFrame.


# spc_cov = list(filter(None, Conversions['ConversionId'].tolist()))


# # Comments for spc_converter:
# # The function checks if ingredient is in the liquid_unit or solid_unit lists. If so, it calls std_converter(qty, uom) to 
# # convert the quantity and UOM to a standardized unit.

# # If uom is not in liquid_unit or solid_unit it checks if ingre is in spc_cov, if it is and the ConvertToUom is equal to grams 
# # then the function applies the factor to the qty argument to convert it to the standardized unit, and returns the result as
# # a tuple containing the converted quantity and uom. If no conversion found, then it calls std_converter(qty, uom)

# # If uom not in liquid_unit or solid_unit and if ingre is not in spc_cov then the function calls std_converter(qty, uom)

# def spc_converter(ingre, qty, uom):
#     if uom in liquid_unit + solid_unit:
#         return std_converter(qty, uom)
#     elif ingre in spc_cov:
#         conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)
#                                     & (Conversions['ConvertToUom'] == 'g')]
#         multiplier = conversion['Multiplier']
#         if multiplier.empty:
#             return std_converter(qty, uom)
#         else: 
#             Qty = float(qty)/float(multiplier)
#             Uom = conversion['ConvertToUom'].values[0]
#             return (Qty, Uom)
#     else:
#         return std_converter(qty, uom)

In [21]:
# # Display first 5 rows
# Conversions.head()

In [22]:
# # Add the specific conversion info from the newly-processed data to a unit conversion database
# Update_Conv = pd.read_csv(os.path.join("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/cleaning/update/Conv_UpdateConv.csv"))
# Update_Conv

In [23]:
# # return dataframe with null values
# # There are no null values
# subset_conv = Update_Conv[Update_Conv["Multiplier"].isna()]
# subset_conv

In [24]:
# # takes a data frame and assigns a new column called Multiplier to the dataframe
# # uses iterrows() to iterate through the rows.Then subset_conv will have a new column named Multiplier with the computed values.
# def assign_multiplier(df):
#     for ind, row in df.iterrows():
#         df.loc[ind, "Multiplier"] = row["ConvertFromQty"] / row["ConvertToQty"]
        
# assign_multiplier(subset_conv)

# # May 8th change: We want to 
# assign_multiplier(Update_Conv)
# #subset_conv

In [25]:
# # Display the first 339 rows and all the columns
# Update_Conv = Update_Conv.iloc[0:339, :]
# Update_Conv

In [26]:
# # convert dataframe to csv file
# Update_Conv = pd.concat([Update_Conv, subset_conv], axis=0)
# Update_Conv.to_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/cleaning/update/Conv_UpdateConv.csv", index=False)

In [27]:
# # retrieves the value in the 'ConversionId' column using Update_Conv.loc[index, 'ConversionId'] 
# # and assigns it to the variable Id. 

# # row: a Series (1-d array in pandas) You can access the individual values of the row using the column headers as keys to the
# # like row['column_name'].

# # if the IDs of Update_Conv are the same IDs as Conversions then we drop it from the Conversions dataframe

# for index, row in Update_Conv.iterrows():
#     Id = Update_Conv.loc[index, 'ConversionId']
#     Conversions.drop(Conversions[Conversions['ConversionId'] == Id].index, inplace = True)

In [28]:
# # combine two data frames, reset the index and remove any duplicates in the concatenated data frame, 
# # and return the resulting data frame.

# # Added comments below on May 8th

# # frames variable is created as a list of two DataFrames, Conversions and Update_Conv.
# # pd.concat() function is used to concatenate the two DataFrames vertically (i.e., stack one on top of the other). 

# # drop_duplicates() method is used to remove any duplicate rows from the concatenated DataFrame based on all columns. 
# # The resulting DataFrame has only unique rows.

# # The resulting DataFrame is assigned back to the variable Conversions,

# # drop = True ensures that the old index is not added as a new column to the dataframe 
# # inplace=False parameter ensures that a new DataFrame is returned rather than modifying 
# # the original Conversions DataFrame in place.

# frames = [Conversions, Update_Conv]
# Conversions = pd.concat(frames).reset_index(drop=True, inplace=False).drop_duplicates()

In [29]:
# # Convert the DataFrame: Conversions into a csv file called Conversions_Added.csv
# path = os.path.join("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/cleaning/Conversions_Added.csv")
# Conversions.to_csv(path, index = False, header = True)

In [30]:
# # Import standard unit conversion information and construct a dataframe
# Std_Unit = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/external/standard_conversions.csv")
# Std_Unit.head()

In [31]:
# # Seperate uoms that converted to 'ml' or 'g'
# # Below we create 2 lists. 
# # list_unit contains list of unit of measurements that are being converted to milliliters 
# # solid_unit contains a list of unit of measurements that are being converted to grams
# # tolist() converts a Pandas Series or an array to a python list. 

# liquid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'ml', 'ConvertFromUom'].tolist()
# solid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'g', 'ConvertFromUom'].tolist()

In [32]:
# # Construct a standard unit converter
# def std_converter(qty, uom):
#     if uom in Std_Unit['ConvertFromUom'].tolist():
#         multiplier = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'Multiplier']
#         Qty = float(qty)*float(multiplier)
#         Uom = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'ConvertToUom'].values[0]
#     else:
#         Qty = qty
#         Uom = uom
#     return (Qty, Uom)

In [33]:
# Convert pounds to grams
# std_converter(0.25,'lb')

In [34]:
# # Construct a unit converter for specific ingredients

# # After this line below, spc_cov contains only the non-empty values from the 'ConversionId' column of the Conversions DataFrame.
# spc_cov = list(filter(None, Conversions['ConversionId'].tolist()))


# # Comments for spc_converter:
# # The function checks if ingredient is in the liquid_unit or solid_unit lists. If so, it calls std_converter(qty, uom) to 
# # convert the quantity and UOM to a standardized unit.

# # If uom is not in liquid_unit or solid_unit it checks if ingre is in spc_cov, if it is and the ConvertToUom is equal to grams 
# # then the function applies the factor to the qty argument to convert it to the standardized unit, and returns the result as
# # a tuple containing the converted quantity and uom. If no conversion found, then it calls std_converter(qty, uom)

# # If uom not in liquid_unit or solid_unit and if ingre is not in spc_cov then the function calls std_converter(qty, uom)

# def spc_converter(ingre, qty, uom):
#     if uom in liquid_unit + solid_unit:
#         return std_converter(qty, uom)
#     elif ingre in spc_cov:
#         conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)
#                                     & (Conversions['ConvertToUom'] == 'g')]
#         multiplier = conversion['Multiplier']
#         if multiplier.empty:
#             return std_converter(qty, uom)
#         else: 
#             Qty = float(qty)/float(multiplier)
#             Uom = conversion['ConvertToUom'].values[0]
#             return (Qty, Uom)
#     else:
#         return std_converter(qty, uom)

In [35]:
# import math
# spc_cov2 = [item for item in spc_cov if not(pd.isnull(item)) == True]
# spc_cov2

**Below I have extracted all unique items with Nonstandard Unit of Measurement in the consolidated report that need to be manually adjusted**

In [36]:
# # Filter out the items whose unit information is unknown 

# # We find the column names
# col_names = list(consolidated_df.columns.values)

# # Create a Items_Nonstd list
# Items_Nonstd = []

# # If the unit of measurement is not grams or ml and ingredient id starts with I and the ingredient is not in ConversionId column of Conversions 
# # then we add it to Items_Nonstd list
# for index, row in consolidated_df.iterrows():
#     Uom = consolidated_df.loc[index,'uom']
#     if Uom not in ['kg', 'lbs', 'oz', 'g', 'ml'] and Uom not in liquid_unit + solid_unit:
#         Dict = {}
#         Dict.update(dict(row))
#         Items_Nonstd.append(Dict)

# # Create a DataFrame from Items_Nonstd list
# Items_Nonstd = pd.DataFrame(Items_Nonstd, columns = col_names)
# # Remove duplicate ingredients of the same properties so that Items_Nonstd has only unique rows. 
# Items_Nonstd.drop_duplicates(subset=['item_num'], inplace=True,)
# Items_Nonstd